In [1]:
import torchvision
import torch
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torchvision import transforms
from torchvision.io import read_image
from art.estimators.classification import PyTorchClassifier
# from art.attacks.evasion import ProjectedGradientDescentPyTorch
from torchattacks import PGD
from art.utils import random_sphere
from art.config import ART_NUMPY_DTYPE
import numpy as np
from torchvision.transforms import v2
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights, vgg19 , VGG19_Weights, resnet18, ResNet18_Weights, densenet121,  densenet121, DenseNet121_Weights, vit_b_16, ViT_B_16_Weights, efficientnet_b2, EfficientNet_B2_Weights,  swin_s, Swin_S_Weights, convnext_base, ConvNeXt_Base_Weights, squeezenet1_0, SqueezeNet1_0_Weights

/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/

In [2]:
import os
os.environ['SLURM_JOBID']

'570820'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
models = [("vit_b_16",vit_b_16, ViT_B_16_Weights),
          ("efficientnet_b0" , efficientnet_b0, EfficientNet_B0_Weights),
          ("swin_s", swin_s, Swin_S_Weights),
          ("densenet121" , densenet121, DenseNet121_Weights),
          ("resnet18" ,resnet18, ResNet18_Weights)]
            # ("convnext_base", convnext_base,ConvNeXt_Base_Weights)]
         # ("squeezenet1_0",squeezenet1_0, SqueezeNet1_0_Weights)]

In [ ]:
x =600

In [ ]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        if x.shape == (3,224,224):
          x = x.unsqueeze(0)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

In [ ]:
import os
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Initialize your custom dataset
# new_dataset = CustomDataset()

In [ ]:
def read_image(image_path):
    return Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

In [ ]:
import pickle
import random
from torch.cuda.amp import autocast
trans = [
    transforms.RandomAffine(degrees=(-2, 2), translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=(224, 224), padding=10),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomAdjustSharpness(sharpness_factor=2,p=1),
    transforms.RandomAutocontrast(1)]


augmentations = transforms.Compose(trans)


In [ ]:
import pickle

batch_size = 100
d = {}
for i , (name_v, model_v, weights_v) in enumerate(models): 
        victim = model_v(weights=weights_v.DEFAULT)
        print("______________________")
        print("Victim: "+ name_v)
    
        S = models.copy()
        S.pop(i)
        victim = model_with_normalization(victim, normalization)
        victim.eval()
        victim = victim.to(device)
        
        with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
            new_data_loader = pickle.load(f)
    
        for j , (name_g,model_g, weights_g) in enumerate(S):
                       row = [0,0,0]
                       print("g: "+ name_g) 
                       g = model_g(weights=weights_g.DEFAULT) 
                       Se = S.copy()
                       Se.pop(j)
                       g = model_with_normalization(g, normalization)
                       g.eval()
                       g = g.to(device)
                       ASR = 0
                       total_l2_score = 0
            
                       for batch , (images, labels) in enumerate(new_data_loader):
                           print(batch)
                           labels =labels.to(device)
                           #PGN
                           PGD_attack = PGD(g, eps=12.75/255, alpha=random.uniform(0.1/255, 0.3/255), steps =random.randint(10, 20), random_start=True)
                           x_p = PGD_attack(images, labels)
                           adv_label_p = victim(x_p).argmax(dim=1)
                           # adv_label_p = adv_label_p.to('cpu')
                           ASR += (labels != adv_label_p).int().sum().item()
                           #print(ASR)
                           l2_score = torch.norm(images - x_p)
                           total_l2_score += l2_score.sum().item()
                           #print(l2_score)
            
                       d[f'{name_v}_{name_g}_ASR'] = ASR/1000
                       d[f'{name_v}_{name_g}_L2'] = total_l2_score/1000
                       print(ASR)

In [ ]:
with open('ranking_simba_baselines/PGD.pkl', 'wb') as f:
     pickle.dump(d, f)

In [10]:
# def pgd_attack(S, images, labels, eps=0.3, alpha=2/255, iters=40) :
#     images = images.to(device)
#     labels = labels.to(device)
#     loss = nn.CrossEntropyLoss()
        
#     ori_images = images.data
        
#     for i in range(iters) :
#         cost = 0
#         images.requires_grad = True
#         for j , (name_g,model_g, weights_g) in enumerate(S):
#             g = model_g(weights=weights_g.DEFAULT)
#             g = model_with_normalization(g, normalization)
#             g.eval()
#             g = g.to(device)
  
#             outputs = g(images)
#             g.zero_grad()
#             cost+= loss(outputs, labels).to(device)
#             del g

        
#         cost.backward()

#         adv_images = images + alpha*images.grad.sign()
#         eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
#         images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    
#     return images

In [11]:
# batch_size = 100

# for i , (name_v, model_v, weights_v) in enumerate(models): 
#     victim = model_v(weights=weights_v.DEFAULT)
#     print("______________________")
#     print("Victim: "+ name_v)

#     S = models.copy()
#     S.pop(i)
#     victim = model_with_normalization(victim, normalization)
#     victim.eval()
#     victim = victim.to(device)
#     total_ASR = 0
#     with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
#         new_data_loader = pickle.load(f)

#     e = 3   
#     for batch , (images, labels) in enumerate(new_data_loader):
#        labels = labels.to(device)
#        x_p = pgd_attack(S, images, labels, eps=e/255, alpha=0.3/255, iters=20)
#        adv_label_p = victim(x_p).argmax(dim=1)
#        # adv_label_p = adv_label_p.to('cpu')
#        ASR = (labels != adv_label_p).int().sum().item()
#        print(ASR)
#        total_ASR+=ASR
#     print(total_ASR)

In [12]:
# from torchvision.models import mnasnet1_0, MNASNet1_0_Weights, efficientnet_b0,regnet_x_800mf, RegNet_X_800MF_Weights, EfficientNet_B0_Weights, vgg19 , VGG19_Weights, resnet18, ResNet18_Weights, densenet121,  densenet121, DenseNet121_Weights, vit_b_16, ViT_B_16_Weights, efficientnet_b2, EfficientNet_B2_Weights,  swin_s, Swin_S_Weights, convnext_base, ConvNeXt_Base_Weights, squeezenet1_0, SqueezeNet1_0_Weights
# S2 = [ ("convnext_base", convnext_base,ConvNeXt_Base_Weights),
#        ("squeezenet1_0",squeezenet1_0, SqueezeNet1_0_Weights),
#       ("mnasnet1_0",mnasnet1_0, MNASNet1_0_Weights),
#       ("regnet_x_800mf", regnet_x_800mf, RegNet_X_800MF_Weights)
#      ]

In [13]:
e=25.5
# x

In [14]:
# #using torchatacks

# batch_size = 100
# for i , (name_v, model_v, weights_v) in enumerate(models): 
#     if i == 0:
#         continue
#     if i>1:
#         break
#     victim = model_v(weights=weights_v.DEFAULT)
#     print("______________________")
#     print("Victim: "+ name_v)

#     S = models.copy()
#     S.pop(i)
#     victim = model_with_normalization(victim, normalization)
#     victim.eval()
#     victim = victim.to(device)
    
#     with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
#         new_data_loader = pickle.load(f)

#     new_data_loader = DataLoader(
#             new_data_loader.dataset, batch_size=batch_size, shuffle=True #[x:x+100]
#         )
    
#     row = [0,0]
#     for j , (name_g,model_g, weights_g) in enumerate(S):
#            if j!=0:
#                 break
#            print("g: "+ name_g) 
#            g = model_g(weights=weights_g.DEFAULT) 
#            Se = S.copy()
#            Se.pop(j)
#            g = model_with_normalization(g, normalization)

#            g.eval()
#            g = g.to(device)
#            ASR = 0
#            for batch , (images, labels) in enumerate(new_data_loader):
#                    labels =labels.to(device)
#                    images = images.to(device)
#                    augmentations_confidences = []
#                    augmentations_success = []
#                    # x_p = PGD(g, eps=e/255, alpha=0.3/255, steps = 20 , random_start=True)

#                    # PGD_attack = PGD(g, eps=e/255, alpha=random.uniform(0.1/255, 0.3/255), steps =random.randint(10, 20), random_start=True)


#                    #  #random pertubations
#                    # x_p = PGD_attack(images, labels)
#                    # adv_label_p = victim(x_p).argmax(dim=1)
#                    # # adv_label_p = adv_label_p.to('cpu')
               
#                    # # adv_label_p = victim(x_p).argmax(dim=1)
#                    # ASR += (labels != adv_label_p).int().sum().item()
#                    # print(ASR)


           
#        #     for _ in tqdm(range(200)):
#        #          victim = victim.to(device)
#        #          # mixing augmentations
#        #          x_a = augmentations(images)
#        #          # images_min = clip_by_tensor(x_a - 12.75 / 255.0, 0.0, 1.0)
#        #          # images_max = clip_by_tensor(x_a + 12.75 / 255.0, 0.0, 1.0)
#        #          x_a = pgd_attack(S2, images, labels, eps=e/255, alpha=0.3/255, iters=20)
#        #          confidance = victim(x_a).softmax(dim=1)
#        #          conf = confidance[torch.arange(confidance.size(0)),labels].detach().cpu().numpy()
#        #          adv_label_a = confidance.argmax(dim=1)
#        #          # adv_label_a = adv_label_a.to('cpu')
#        #          del confidance

#        #          torch.cuda.empty_cache()
#        #          # del PGD_attack
#        #          victim = victim.to('cpu')

#        #          # average_confidences_p =  np.zeros(batch_size, dtype=np.float32)
#        #          average_confidences_a =  np.zeros(batch_size, dtype=np.float32)
#        #          # average_confidences_o =  np.zeros(batch_size, dtype=np.float32)
#        #          for n , model_s, weights_s in S:

#        #              model = model_s(weights=weights_s.DEFAULT)
#        #              model = model.to(device)
#        #              model = model_with_normalization(model, normalization)
#        #              model.eval()

#        #              outputs_a = model(x_a).softmax(dim=1)
#        #              confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
#        #              average_confidences_a += confidences_a
#        #              del outputs_a,confidences_a
#        #              del model


#        #              torch.cuda.empty_cache()
#        #          del x_a#x_p,x_a,x_o

#        #          average_confidences_a /= len(S)
#        #          equality_tensor_a = (labels != adv_label_a).int() 

#        #          augmentations_confidences.append(average_confidences_a)
#        #          augmentations_success.append(equality_tensor_a)
#        #          torch.cuda.empty_cache()
       
#        #     merged_results_a = [
#        #                          tuple(
#        #                              zip(
#        #                                  np.array(augmentations_confidences)[:, i],
#        #                                  torch.stack(augmentations_success).cpu().numpy()[:, i]
#        #                              )
#        #                          ) 
#        #                          for i in range(batch_size)
#        #                      ]


#        #     # merged_results_o = [tuple(zip(np.array(augmentation_confidences)[:, i], torch.stack(augmentation_success).numpy()[:, i]))
#        #     #      for i in range(batch_size)]

#        #      # Sort the merged results based on the mean confidence for each image
#        #     # sorted_results_p = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_p]
#        #     #sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]
#        #     # sorted_results_o = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_o]

#        #     sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]

        

#        #     best_result = 0
#        #     random_result = 0 
#        #     best_v = 0
#        #     for sublist in sorted_results_a:

#        #          best_result += sublist[0][1]
#        #          random_result += random.choice(sublist)[1]


#        #     row[0]+=random_result
#        #     row[1]+=best_result
#        #     # row[6]+=best_v
#        #     print(row) 
#        #     del images, labels
#        #     torch.cuda.empty_cache()
#        # print(row)     

In [15]:
# models = ["vit_b_16","efficientnet_b0","swin_s","densenet121","resnet18"]
# eps = [2, 4, 6, 8, 10, 12.75 , 14, 16, 18, 20]

In [16]:
# for e in eps:
#     table_e = []
#     for i , name_v in enumerate(models):
#         S = models.copy()
#         S.pop(i)        
#         for j , name_g in enumerate(S):
#             # print(j)
#             if j==3:
#                  with open(name_v+'_'+name_g+'_'+str(e)+'.pkl', 'rb') as f:
#                                  results = pickle.load(f)
#                                  table_e+=results

#     table_e = [[item / 1000 for item in sublist] for sublist in table_e]

#     with open('tables/table_'+str(e)+'.pkl', 'wb') as f:
#                      pickle.dump(table_e, f)

In [17]:
import random
batch_size =100
import warnings 
warnings.filterwarnings('ignore', category=Warning)

for i , (name_v, model_v, weights_v) in enumerate(models): 
    if i == 0:
        continue
    if i>1:
        break
    victim = model_v(weights=weights_v.DEFAULT)
    print("______________________")
    print("Victim: "+ name_v)

    S = models.copy()
    S.pop(i)
    victim = model_with_normalization(victim, normalization)
    victim.eval()
    victim = victim.to(device)
    
    with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
        new_data_loader = pickle.load(f)

    new_data_loader = DataLoader(
            new_data_loader.dataset[x:x+100], batch_size=batch_size, shuffle=True #[x:x+100]
        )

    for j , (name_g,model_g, weights_g) in enumerate(S):
                   row = [0,0,0,0,0,0,0]
                   print("g: "+ name_g)
                   if j>0:
                       break
                    
                   g = model_g(weights=weights_g.DEFAULT) 
                   Se = S.copy()
                   Se.pop(j)
                   g = model_with_normalization(g, normalization)

                   g.eval()
                   g = g.to(device)
                   for batch , (images, labels) in enumerate(new_data_loader):
                       perturbation_confidences = []
                       perturbation_success = []

                       augmentations_confidences = []
                       augmentations_success = []

                       augmentation_confidences = []
                       augmentation_success = [] 

                       confidences = []
                       print(batch)
                       for _ in tqdm(range(200)):
                            victim = victim.to(device)
                            PGD_attack = PGD(g, eps=e/255, alpha=random.uniform(0.1/255, 0.3/255), steps =random.randint(10, 20), random_start=True)


                            # #random pertubations
                            # x_p = PGD_attack(images, labels)
                            # adv_label_p = victim(x_p).argmax(dim=1)
                            # adv_label_p = adv_label_p.to('cpu')

                            #mixing augmentations
                            x_a = augmentations(images)
                            x_a = PGD_attack(x_a, labels)
                            confidance = victim(x_a).softmax(dim=1)
                            conf = confidance[torch.arange(confidance.size(0)),labels].detach().cpu().numpy()
                            #conf = conf.to('cpu')

                            adv_label_a = confidance.argmax(dim=1)
                            adv_label_a = adv_label_a.to('cpu')
                            del confidance

                            # #one augmentation
                            # x_o = random.choice(trans)(images)
                            # x_o = PGD_attack(x_o, labels)
                            # adv_label_o = victim(x_o).argmax(dim=1)
                            # adv_label_o = adv_label_o.to('cpu')

                            torch.cuda.empty_cache()
                            del PGD_attack
                            victim = victim.to('cpu')

                            # average_confidences_p =  np.zeros(batch_size, dtype=np.float32)
                            average_confidences_a =  np.zeros(batch_size, dtype=np.float32)
                            # average_confidences_o =  np.zeros(batch_size, dtype=np.float32)
                            for n , model_s, weights_s in Se:

                                model = model_s(weights=weights_s.DEFAULT)
                                model = model.to(device)
                                model = model_with_normalization(model, normalization)
                                model.eval()

                                # outputs_p = model(x_p).softmax(dim=1)
                                # confidences_p = outputs_p[torch.arange(outputs_p.size(0)),labels].detach().cpu().numpy()
                                # average_confidences_p += confidences_p
                                # del outputs_p,confidences_p


                                outputs_a = model(x_a).softmax(dim=1)
                                confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
                                average_confidences_a += confidences_a
                                del outputs_a,confidences_a


                                # outputs_o = model(x_o).softmax(dim=1)
                                # confidences_o = outputs_o[torch.arange(outputs_o.size(0)),labels].detach().cpu().numpy()                            
                                # average_confidences_o += confidences_o

                                # outputs_o = outputs_o.to('cpu')
                                # model = model.to('cpu')
                                # del outputs_o,confidences_o,model
                                # # del confidences_o
                                # del model


                                torch.cuda.empty_cache()
                            # del x_p,x_a,x_o



                            # average_confidences_p /= len(Se)
                            average_confidences_a /= len(Se)
                            # average_confidences_o /= len(Se)

                            # equality_tensor_p = (labels != adv_label_p).int()
                            equality_tensor_a = (labels != adv_label_a).int() 
                            # equality_tensor_o = (labels != adv_label_o).int()



                            confidences.append(conf)

                            # perturbation_confidences.append(average_confidences_p)
                            # perturbation_success.append(equality_tensor_p)
                            augmentations_confidences.append(average_confidences_a)
                            augmentations_success.append(equality_tensor_a)
                            # augmentation_confidences.append(average_confidences_o)
                            # augmentation_success.append(equality_tensor_o)

                            torch.cuda.empty_cache()
                       # merged_results_p = [tuple(zip(np.array(perturbation_confidences)[:, i], torch.stack(perturbation_success).numpy()[:, i]))
                       #      for i in range(batch_size)]

                       # merged_results_a = [tuple(zip(np.array(augmentations_confidences)[:, i], torch.stack(augmentations_success).numpy()[:, i]))
                       #      for i in range(batch_size)]
                       merged_results_a = [tuple(zip(np.array(augmentations_confidences)[:, i], np.array(confidences)[:, i], torch.stack(augmentations_success).numpy()[:, i])) for i in range(batch_size)]



                       # merged_results_o = [tuple(zip(np.array(augmentation_confidences)[:, i], torch.stack(augmentation_success).numpy()[:, i]))
                       #      for i in range(batch_size)]

                        # Sort the merged results based on the mean confidence for each image
                       # sorted_results_p = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_p]
                       #sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]
                       # sorted_results_o = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_o]

                       sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]

                       
                       # best_result = 0
                       # random_result = 0
                       # for sublist in sorted_results_p:
                       #      # Increment the best result based on the first item's binary value
                       #      best_result += sublist[0][1]
                       #      # Calculate the sum of random items in each sublist
                       #      random_result += random.choice(sublist)[1]

                       # row[0]+=random_result
                       # row[1]+=best_result

                       # best_result = 0
                       # random_result = 0 
                       # for sublist in sorted_results_o:
                       #      # Increment the best result based on the first item's binary value
                       #      best_result += sublist[0][1]
                       #      # Calculate the sum of random items in each sublist
                       #      random_result += random.choice(sublist)[1]
                       # row[2]+=random_result
                       # row[3]+=best_result


                       best_result = 0
                       random_result = 0 
                       best_v = 0
                       for sublist in sorted_results_a:

                            best_result += sublist[0][2]
                            # best_v += min(sublist, key=lambda x: x[1])[2]
                            # Increment the best result based on the first item's binary value
                            #best_result += sublist[0][1]
                            # Calculate the sum of random items in each sublist
                            random_result += random.choice(sublist)[2]

                       row[4]+=random_result
                       row[5]+=best_result
                       # row[6]+=best_v
                       print(row) 
                       # upperB[(name_v,name_g)] += best_v  
                       # print(results_a)     
                       # print(upperB)  
                       del images, labels
                       torch.cuda.empty_cache()
                        

                   del g
                   with open('BTA_PEAS/'+name_v+'_'+name_g+'_'+str(e)+'_'+str(x)+ '.pkl', 'wb') as f:
                     pickle.dump(row, f)
                    

______________________
Victim: efficientnet_b0
g: vit_b_16
0


100%|██████████| 200/200 [29:12<00:00,  8.76s/it]

[0, 0, 0, 0, 50, 65, 0]
g: swin_s
